In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
import random
import pyautogui
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [ ]:
def configurar_driver():
    chrome_options = webdriver.ChromeOptions()
    options = chrome_options
    return webdriver.Chrome(options=options)

def abrir_pagina(driver, url):
    driver.get(url)
    driver.maximize_window()
    time.sleep(15)

def extraer_enlaces_videos(driver, cantidad):
    enlaces_videos = []
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    div_videos = soup.find_all('div', class_='css-1soki6-DivItemContainerForSearch')
    for div_video in div_videos[:cantidad]:  # Limitar la cantidad de enlaces a extraer
        enlace = div_video.find('a')['href']
        enlaces_videos.append(enlace)
    return enlaces_videos

def extraccion_link(palabra, cantidad):
    driver = configurar_driver()  # Inicializar el controlador una vez fuera del bucle
    video_url = "https://www.tiktok.com/"  # URL del primer video
    abrir_pagina(driver, video_url)
    time.sleep(5)
    performActions(698, 188, '#milei')
    cantidad = 10
    time.sleep(5)
    enlaces = extraer_enlaces_videos(driver, cantidad)
    driver.quit()
    return enlaces

def type_with_random_delay(text):
    for char in text:
        pyautogui.typewrite(char)
        time.sleep(random.uniform(0.1, 0.3))  # Ajusta los valores según sea necesario

def performActions(x_pos, y_pos, input_string=False):
    move_mouse_random_straight_line(x_pos, y_pos)
    time.sleep(random.random())
    
    # Perform a mouse click at the target location
    pyautogui.click(x_pos, y_pos)
    
    if input_string:
        type_with_random_delay(input_string)
        time.sleep(random.random())
        pyautogui.press("enter")

def move_mouse_and_click(x_pos, y_pos, input_string=False):
    move_mouse_random_straight_line(x_pos, y_pos)
    time.sleep(random.random())
    pyautogui.press("enter")
    
    # Perform a mouse click at the target location
    pyautogui.click(x_pos, y_pos)

def move_mouse_random_straight_line(target_x, target_y):
    current_x, current_y = pyautogui.position()

    deviation_range = 2

    # Calculate the distance between current and target position
    distance = ((target_x - current_x) ** 2 + (target_y - current_y) ** 2) ** 0.5

    # Number of steps to take in the straight line trajectory
    num_steps = int(distance / 20)  # Ajusta el tamaño del paso según sea necesario
    if num_steps == 0:
        num_steps = 1  # Ajusta el tamaño del paso según sea necesario

    # Calculate step size in x and y directions
    step_x = (target_x - current_x) / num_steps
    step_y = (target_y - current_y) / num_steps

    # To stop the process when is close enough
    stop_process = 0

    # Move the mouse in a straight line trajectory
    while current_x != target_x or current_y != target_y:
        current_x, current_y = pyautogui.position()

        # Calculate the distance between current and target position
        distance = ((target_x - current_x) ** 2 + (target_y - current_y) ** 2) ** 0.5

        # Number of steps to take in the straight line trajectory
        num_steps = int(distance / 20)  # Ajusta el tamaño del paso según sea necesario
        if num_steps == 0:
            # To stop the process when is close enough
            stop_process = stop_process + 1
            num_steps = 1  # Ajusta el tamaño del paso según sea necesario
            deviation_range = 1

        # Calculate step size in x and y directions
        step_x = (target_x - current_x) / num_steps
        step_y = (target_y - current_y) / num_steps

        current_x += step_x + random.randint(-deviation_range, deviation_range)
        current_y += step_y + random.randint(-deviation_range, deviation_range)
        pyautogui.moveTo(current_x, current_y, duration=0.01)

def obtener_numero_comentarios(driver):
    return driver.find_elements(By.CSS_SELECTOR, 'div.css-1i7ohvi-DivCommentItemContainer')

def hacer_scroll_cargar_comentarios2(driver, initial_comments):
    """
    Esta función carga comentarios en una página web evitando la carga infinita de videos.

    Args:
        driver (webdriver): El controlador del navegador web.
        initial_comments (list): Una lista de elementos de comentarios iniciales (opcional).

    Returns:
        bool: True si se han cargado nuevos comentarios, False en caso contrario.
    """

    previous_comment_count = 0  # Número de comentarios encontrados previamente
    max_iterations = 10  # Máximo número de iteraciones para evitar bucles infinitos

    for _ in range(max_iterations):
        # Encontrar el primer elemento de comentario para rastrear cambios
        primer_comentario = driver.find_element(By.CSS_SELECTOR, 'div.css-1i7ohvi-DivCommentItemContainer')

        # Desplazarse hacia abajo y esperar nuevos comentarios
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.css-1i7ohvi-DivCommentItemContainer')))
        time.sleep(2)

        # Contar la cantidad actual de comentarios
        cantidad_comentarios_actual = len(driver.find_elements(By.CSS_SELECTOR, 'div.css-1i7ohvi-DivCommentItemContainer'))

        # Verificar si se han cargado nuevos comentarios
        if cantidad_comentarios_actual == previous_comment_count:
            break  # Salir del bucle si no hay nuevos comentarios

        previous_comment_count = cantidad_comentarios_actual

    # Extraer comentarios usando información obtenida (primer_comentario, cantidad_comentarios_actual)
    # ... (código para extraer comentarios)

    return True  # Se han cargado nuevos comentarios

def escribir_csv2(comentario_containers):
    with open('comentarios_tiktok.csv', mode='a', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)

        for comentario_container in comentario_containers:
            nombre_usuario, fecha, likes, comentario_texto = obtener_info_comentario(comentario_container)
            writer.writerow([nombre_usuario, fecha, likes, comentario_texto])
            print(f"Nombre de usuario: {nombre_usuario}")
            print(f"Fecha del comentario: {fecha}")
            print(f"Cantidad de likes: {likes}")
            print(f"Comentario: {comentario_texto}")
            print("------")

            respuesta_containers = comentario_container.find_all('div', class_='css-1i7ohvi-DivCommentItemContainer')
            for respuesta_container in respuesta_containers:
                nombre_usuario_resp, fecha_resp, likes_resp, comentario_texto_resp = obtener_info_comentario(respuesta_container)
                writer.writerow([nombre_usuario_resp, fecha_resp, likes_resp, comentario_texto_resp])
                print(f"Respuesta - Nombre de usuario: {nombre_usuario_resp}")
                print(f"Respuesta - Fecha del comentario: {fecha_resp}")
                print(f"Respuesta - Cantidad de likes: {likes_resp}")
                print(f"Respuesta - Comentario: {comentario_texto_resp}")
                print("------")

def obtener_info_comentario(comentario_container):
    nombre_usuario_tag = comentario_container.find('a', href=True)
    if nombre_usuario_tag:
        nombre_usuario_href = nombre_usuario_tag.get('href')
        nombre_usuario = nombre_usuario_href.split('/')[-1]
    else:
        nombre_usuario = "No se encontró el nombre de usuario"

    fecha_tag = comentario_container.find('span', class_='css-1esugaz-SpanCreatedTime e1g2efjf8')
    fecha = fecha_tag.text if fecha_tag else "No se encontró la fecha"

    like_tag = comentario_container.find('span', class_='css-gb2mrc-SpanCount ezxoskx3')
    likes = like_tag.text if like_tag else "No se encontró la cantidad de likes"

    comentario_tag = comentario_container.find('p', class_='css-xm2h10-PCommentText e1g2efjf6')
    comentario_texto = comentario_tag.text if comentario_tag else "No se encontró el comentario"

    return nombre_usuario, fecha, likes, comentario_texto

def main(enlaces, iteraciones=2):
    for video_url in enlaces:
        driver = configurar_driver()
        abrir_pagina(driver, video_url)

        initial_comments = obtener_numero_comentarios(driver)

        while True:
            if hacer_scroll_cargar_comentarios2(driver, initial_comments):
                break

        html_code = driver.page_source
        soup = BeautifulSoup(html_code, 'html.parser')
        comentario_containers = soup.find_all('div', class_='css-1i7ohvi-DivCommentItemContainer')

        escribir_csv2(comentario_containers)

        print(f"La información del video {video_url} se ha guardado en 'comentarios_tiktok.csv'")

        # Simular presionar la tecla de flecha hacia abajo para cambiar al siguiente video
        #actions = webdriver.ActionChains(driver)
        #actions.send_keys(Keys.ARROW_DOWN).perform()
        time.sleep(5)

    driver.quit()


In [ ]:
# obtener enlaces
enlaces = extraccion_link('#milei', 10)
print(enlaces)

In [ ]:
#recorrer enlaces
main(enlaces)

In [ ]:
#CSV to json
import pandas as pd

df = pd.read_csv("comentarios_tiktok.csv")
df.to_json("comentarios_tiktok.json")